# Differential expression analysis


You have run the nf-core/rnaseq pipeline and checked the first quality control metrics of your fastq files. This was, however, only the primary analysis and we want to take it further.

Due to the computational demand of the pipeline, you only ran the pipeline on two of the 16 samples in the study yesterday. We provide you an essential output of nf-core/rnaseq pipeline in the `data` folder: It contains the combined epression matrix as produced by Salmon, which provides transcript levels for each gene (rows) and each sample (columns).


We would now like to understand exactly the difference between the expression in our groups of mice. 
Which pipeline would you use for this?

I would use the differentialabundance pipeline again. 

Have a close look at the pipeline's "Usage" page on the [nf-core docs](nf-co.re). You will need to create a samplesheet (based on the column names in the provided matrix).

The samplesheet should look like this: 
sample,fastq_1,fastq_2,condition,replicate,batch
CONTROL_REP1,AEG588A1_S1_L002_R1_001.fastq.gz,AEG588A1_S1_L002_R2_001.fastq.gz,control,1,A
CONTROL_REP2,AEG588A1_S1_L003_R1_001.fastq.gz,AEG588A1_S1_L003_R2_001.fastq.gz,control,2,B
CONTROL_REP3,AEG588A1_S1_L004_R1_001.fastq.gz,AEG588A1_S1_L004_R2_001.fastq.gz,control,3,A
TREATED_REP1,AEG588A2_S1_L002_R1_001.fastq.gz,AEG588A2_S1_L002_R2_001.fastq.gz,treated,1,B
TREATED_REP2,AEG588A2_S1_L003_R1_001.fastq.gz,AEG588A2_S1_L003_R2_001.fastq.gz,treated,2,A
TREATED_REP3,AEG588A2_S1_L004_R1_001.fastq.gz,AEG588A2_S1_L004_R2_001.fastq.gz,treated,3,B

In [7]:
import pandas as pd

columns = pd.read_csv("data/salmon.merged.gene_counts.tsv", sep="\t", nrows=0).columns.tolist()

samplesheet = pd.DataFrame()

samplesheet["sample"] = columns[2:]
samplesheet["condition"] = samplesheet["sample"].str.rsplit("_", n=1).str[0]
samplesheet["replicate"] = samplesheet["sample"].str.rsplit("_", n=1).str[1]
samplesheet.to_csv("samplesheet.csv", header=True, index=False)

Please paste here the command you used. You may need to inspect the provided expression matrix more closely and create additional files, like a samplesheet (based on the column names) or a contrast file (there happens to also be one in `data/` that you can use).

In [4]:
!nextflow run nf-core/differentialabundance -r 1.5.0 --input samplesheet.csv --contrasts data/contrasts.csv --matrix data/salmon.merged.gene_counts.tsv --outdir differential  -profile rnaseq,docker --max_memory "5GB"

Nextflow 25.04.7 is available - Please consider updating your version to it

 N E X T F L O W   ~  version 25.04.0

Launching `https://github.com/nf-core/differentialabundance` [mighty_shannon] DSL2 - revision: 3dd360fed0 [1.5.0]

WARN: Access to undefined parameter `monochromeLogs` -- Initialise it to a default value eg. `params.monochromeLogs = some_value`


------------------------------------------------------
                                        ,--./,-.
        ___     __   __   __   ___     /,-._.--~'
  |\ | |__  __ /  ` /  \ |__) |__         }  {
  | \| |       \__, \__/ |  \ |___     \`-._,-`-,
                                        `._,._,'
  nf-core/differentialabundance v1.5.0-g3dd360f
------------------------------------------------------
Core Nextflow options
  revision                    : 1.5.0
  runName                     : mighty_shannon
  containerEngine             : docker
  container                   : [RMARKDOWNNOTEBOOK:biocontainers/r-shinyngs:1.8.8--r43hd

Explain all the parameters you set and why you set them in this way. If you used or created additional files as input, explain what they are used for.

First I prepared the samplesheet, where I used the sample ID, the condition and the replicate number. 

The contrast file was provided. 

For the command I used: 
- !nextflow run nf-core/differentialabundance: to run it
- -r 1.5.0: This is for the version of the pipeline
- --input samplesheet.csv: The samplesheet I created
- --contrasts data/contrasts.csv: The contrast file that was provided
- --matrix data/salmon.merged.gene_counts.tsv: The matrix that was provided
- --outdir differential: The final folder I woudl like to save my data in 
- -profile rnaseq,docker: Connect it to the docker to use
- --max_memory "5GB": Set the max memory so that it runs through. 

In between I also used this parameter: --transcript_length_matrix 'data/rnaseq-out-small/star_salmon/salmon.merged.gene_lengths.tsv', but it did not worked so I deleted it and then it worked.


What were the outputs of the pipeline?

In the differential folder are multiple folders with outputs: 
- other
- pipeline_info
- plots
- reports
- shinyngs_app
- tables

In [2]:
#!TODO

Would you exclude any samples? If yes, which and why?

I would exclude SNI_Sal_2 and maybe also SNI_Sal_4, as these two samples mainly contribute to the variance of PC1 (22,7%). Without these two samples the picture of the others samples could be different, and maybe would leave more room for interpretation.

How many genes were differentially expressed in each contrast? Does this confirm what the paper mentions?

In the report there are two tables given. One for adjusted p-values, ther I have the following values: 

| Comparison                          | up | down |
|-------------------------------------|----|------|
| SNI_oxy versus SNI_Sal in condition |  1 |   17 |
| Sham_oxy versus Sham_Sal in condition |  7 |    0 |


And the second one with unadjusted p-values: 

| Comparison                          | up | down |
|-------------------------------------|----|------|
| SNI_oxy versus SNI_Sal in condition |  1 |   26 |
| Sham_oxy versus Sham_Sal in condition |  9 |    1 |


In the paper they are comparing also the other conditions (e.g SNI-Oxy versus Sham-Sal) with each other and also dividing the different brain region they took tissue from. 


The paper mentions differentially expressed genes in three brain regions : the NAc, mPFC and VTA. Briefly explain what these 3 regions are.

NAc (nucleus accumbens): It is a region in the basal forebrain rostral to the preoptic area of the hypothalamus and is part of the reward system, plays an important role in processing rewarding stimuli, reinforcing stimuli (e.g., food and water), and those which are both rewarding and reinforcing (addictive drugs, sex, and exercise).

mPFC (medial prefrontal cortex): It is a brain region critical for social cognition, decision-making, and emotion regulation, integrating information from other brain areas and modulating various cognitive functions and behaviors

VTA (ventral tegmental area): The VTA dopamine neurons, serve several functions in the reward system, motivation, cognition, and drug addiction, and may be the focus of several psychiatric disorders

Is there anyway from the paper and the material and methods for us to know which genes are included in these regions?

Once you have your list of differentially expressed genes, do you think just communicating those to the biologists would be sufficient? What does the publication state?

Please reproduce the Venn Diagram from Figure 3, not taking into account the brain regions but just the contrasts mentionned.

In [ ]:
# Adapte contrast file to get the same comparisons from the Venn-Diagramm and exclude the two outliers. 
samplesheet2 = samplesheet.copy()
samplesheet2 = samplesheet[samplesheet["sample"] != "SNI_Sal_4"]
samplesheet2 = samplesheet2[samplesheet2["sample"] != "SNI_Sal_2"]
samplesheet2.to_csv("samplesheet2.csv", header=True, index=False)

contrast2 = pd.read_csv("data/contrasts.csv")
contrast2.loc[len(contrast2)] = ["condition_contorl_treated_3", "condition", "Sham_Sal", "SNI_Sal"]
contrast2["reference"] = "Sham_Sal"
contrast2.to_csv("data/contrast2.csv", header=True, index=False)


In [18]:
!nextflow run nf-core/differentialabundance -r 1.5.0 --input samplesheet.csv --contrasts data/contrasts2.csv --matrix data/salmon.merged.gene_counts.tsv --outdir differential_adapt  -profile rnaseq,docker --max_memory "5GB"

Nextflow 25.04.7 is available - Please consider updating your version to it

 N E X T F L O W   ~  version 25.04.0

Launching `https://github.com/nf-core/differentialabundance` [peaceful_newton] DSL2 - revision: 3dd360fed0 [1.5.0]

WARN: Access to undefined parameter `monochromeLogs` -- Initialise it to a default value eg. `params.monochromeLogs = some_value`


------------------------------------------------------
                                        ,--./,-.
        ___     __   __   __   ___     /,-._.--~'
  |\ | |__  __ /  ` /  \ |__) |__         }  {
  | \| |       \__, \__/ |  \ |___     \`-._,-`-,
                                        `._,._,'
  nf-core/differentialabundance v1.5.0-g3dd360f
------------------------------------------------------
Core Nextflow options
  revision                    : 1.5.0
  runName                     : peaceful_newton
  containerEngine             : docker
  container                   : [RMARKDOWNNOTEBOOK:biocontainers/r-shinyngs:1.8.8--r43